In [34]:
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import time
from wordcloud import WordCloud, STOPWORDS
from IPython.display import display, HTML
import plotly.graph_objects as go
import re
# Natural Language Tool Kit
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter
import cufflinks as cf

cf.go_offline()

In [76]:
import pandas as pd

# Load CSV file into a dataframe
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
tweets = pd.read_csv("tweets.csv")


In [36]:
test_df.shape

(3263, 4)

In [37]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [82]:
df = pd.concat([test_df, tweets, train_df],    # Combine vertically
                          ignore_index = True,
                          sort = False)
df = df[['text', 'target']].copy()

In [83]:
from nltk.corpus import wordnet 

In [84]:
import nltk
#nltk.download('omw-1.4')

related_words = []
for syn in wordnet.synsets("earthquake"):
    for lemma in syn.lemmas():
        related_words.append(lemma.name())
    for hyp in syn.hypernyms():
        for lemma in hyp.lemmas():
            related_words.append(lemma.name())
    for hyp in syn.hyponyms():
        for lemma in hyp.lemmas():
            related_words.append(lemma.name())

print(related_words)
type(related_words)

['earthquake', 'quake', 'temblor', 'seism', 'geological_phenomenon', 'seaquake', 'submarine_earthquake', 'shock', 'seismic_disturbance', 'tremor', 'earth_tremor', 'microseism', 'earthquake', 'disturbance', 'disruption', 'commotion', 'flutter', 'hurly_burly', 'to-do', 'hoo-ha', 'hoo-hah', 'kerfuffle']


list

In [85]:
# Create a new column in the dataset to store whether a tweet is related to an earthquake
df["earthquake"] = df["text"].apply(lambda x: any(word in x for word in related_words))


In [86]:
df.head()

,text,target,earthquake
0,Just happened a terrible car crash,NaN,False
1,"Heard about #earthquake is different cities, stay safe everyone.",NaN,True
2,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",NaN,False
3,Apocalypse lighting. #Spokane #wildfires,NaN,False
4,Typhoon Soudelor kills 28 in China and Taiwan,NaN,False


In [87]:
pd.set_option('display.max_colwidth', None)

,text,target,earthquake
1,"Heard about #earthquake is different cities, stay safe everyone.",NaN,True
5,We're shaking...It's an earthquake,NaN,True
44,Please like and share our new page for our Indoor Trampoline Park Aftershock opening this fall!! http://t.co/UgXhHErrxS,NaN,True
45,@bxckylynch foi no ROH Aftershock: Las Vegas procura no pirate bay que tem,NaN,True
46,Schoolboy ÛÒ Aftershock (Original Mix)\nExcision &amp; Skism ÛÒ SEXisM (Far Too Loud Remix)\nFirebeatz Schella ÛÒ Dear New... http://t.co/JQLzUA6YzQ,NaN,True
...,...,...,...
4256,@esskayen Im still in shock,NaN,True
4257,@rhaplord Southern Africa has some buried ancient history that will soon shock mankind,NaN,True
4258,"A 3.1 magnitude #earthquake occured at 141 km SE of Akutan, Alaska. See the full report at: https://t.co/6G96eGwSKY https://t.co/iSZJ2OCMqR",NaN,True
4261,"A 2.6 magnitude #earthquake occured at 50 km W of Mentone, Texas. See the full report at: https://t.co/u05byRkaMP https://t.co/kiT02EbgyE",NaN,True


In [88]:
df["earthquake"].value_counts()

False    11030
True       846
Name: earthquake, dtype: int64

In [98]:
# Create a dataframe with target = 1 or 2 and earthquake = True
df_train = df.loc[(df['target'].isin([1])) & (df['earthquake'] == True)]

# Create a dataframe with target = NaN and earthquake = False
df_test = df.loc[(df['target'].isna()) & (df['earthquake'] == True)]

In [100]:
df_test = df_test.drop(columns=['target'])

In [101]:
df_test.head()

,text,earthquake
1,"Heard about #earthquake is different cities, stay safe everyone.",True
5,We're shaking...It's an earthquake,True
44,Please like and share our new page for our Indoor Trampoline Park Aftershock opening this fall!! http://t.co/UgXhHErrxS,True
45,@bxckylynch foi no ROH Aftershock: Las Vegas procura no pirate bay que tem,True
46,Schoolboy ÛÒ Aftershock (Original Mix)\nExcision &amp; Skism ÛÒ SEXisM (Far Too Loud Remix)\nFirebeatz Schella ÛÒ Dear New... http://t.co/JQLzUA6YzQ,True


In [102]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import * 
from tensorflow.keras.layers import *


/Users/rafaellaporto/opt/anaconda3/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning:

Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.

/Users/rafaellaporto/opt/anaconda3/lib/python3.9/site-packages/transformers/generation_tf_utils.py:24: FutureWarning:

Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.



TweetNormalizer
From:https://github.com/VinAIResearch/BERTweet/blob/master/TweetNormalizer.py

In [104]:
#twitter normalizer

from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

BERTweetTokenizer
From: https://www.kaggle.com/christofhenkel/setup-tokenizer

In [108]:
from types import SimpleNamespace
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary


class BERTweetTokenizer():
    
    def __init__(self,pretrained_path):
        bpe_dir = os.path.join(pretrained_path,"bpe.codes")
        vocab_dir = os.path.join(pretrained_path,"dict.txt")
        
        self.bpe = fastBPE(SimpleNamespace(bpe_codes= bpe_dir))
        self.vocab = Dictionary()
        self.vocab.add_from_file(vocab_dir)
        
        self.cls_token_id = 0
        self.pad_token_id = 1
        self.sep_token_id = 2
        
        self.pad_token = '<pad>'
        self.cls_token = '<s>'
        self.sep_token = '</s>'
        
    def bpe_encode(self,text):
        return self.bpe.encode(text)
    
    def encode(self,text,add_special_tokens=False):
        subwords = self.bpe.encode(text)
        input_ids = self.vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
        return input_ids
    
    def tokenize(self,text):
        return self.bpe_encode(text).split()
    
    def convert_tokens_to_ids(self,tokens):
        input_ids = self.vocab.encode_line(' '.join(tokens), append_eos=False, add_if_not_exist=False).long().tolist()
        return input_ids
    
    #from: https://www.kaggle.com/nandhuelan/bertweet-first-look
    def decode_id(self,id):
        return self.vocab.string(id, bpe_symbol = '@@')
    
    def decode_id_nospace(self,id):
        return self.vocab.string(id, bpe_symbol = '@@ ')



In [ ]:
tokenizer = BERTweetTokenizer('/kaggle/input/bertweet-base-transformers')